In [2]:
import gym
env = gym.make("Taxi-v3", render_mode="human")
env = env.unwrapped

In [2]:
#遊戲重新開機
print(env.reset())
env.render()

(194, {'prob': 1.0, 'action_mask': array([1, 1, 0, 1, 0, 0], dtype=int8)})


In [9]:
#遊戲關機
env.close()

In [3]:
print(env.action_space.n) # 6
print(env.observation_space.n) # 500

print(len(env.P)) # 500

print(env.P[0]) 
# {
#  0: [(1.0, 100, -1, False)],
#  1: [(1.0, 0, -1, False)],
#  2: [(1.0, 20, -1, False)],
#  3: [(1.0, 0, -1, False)],
#  4: [(1.0, 16, -1, False)],
#  5: [(1.0, 0, -10, False)]
# }

6
500
500
{0: [(1.0, 100, -1, False)], 1: [(1.0, 0, -1, False)], 2: [(1.0, 20, -1, False)], 3: [(1.0, 0, -1, False)], 4: [(1.0, 16, -1, False)], 5: [(1.0, 0, -10, False)]}


In [98]:
env.step(1) # choose action 0
#state, reward, terminal state, info

(375,
 -1,
 False,
 False,
 {'prob': 1.0, 'action_mask': array([1, 1, 1, 0, 0, 0], dtype=int8)})

# Policy Iteration

In [28]:
import numpy as np
import gym

def policy_evaluation(V, policy, gamma, theta):
    while True:
        delta = 0
        for s in range(env.observation_space.n):
            v = 0
            for action, action_prob in enumerate(policy[s]):
                for prob, next_state, reward, done in env.P[s][action]:
                    if done:
                        v += action_prob * prob * reward
                    else:
                        v += (action_prob * prob * (reward + gamma * V[next_state]))
            delta = max(delta, np.abs(v - V[s]))
            V[s] = v
        if delta < theta:
            break

def policy_improvent(V, policy, gamma):
    policy_stable = True
    for s in range(env.observation_space.n):
        old_policy = policy[s].copy()
        q_s = np.zeros(env.action_space.n)
        for a in range(env.action_space.n):
            for prob, next_state, reward, done in env.P[s][a]:
                if done:
                    q_s[a] += prob * reward
                else:
                    q_s[a] += (prob * (reward + gamma * V[next_state]))
        best_action = np.argmax(q_s)
        policy[s] = np.eye(env.action_space.n)[best_action]
        if np.any(old_policy != policy[s]):
            policy_stable = False
    return policy_stable

def policy_iteration(gamma = 0.99, max_iteration = 10000, theta = 0.0001):
    V = np.zeros(env.observation_space.n)
    policy = np.ones([env.observation_space.n, env.action_space.n]) / env.action_space.n
    for i in range(max_iteration):  
        policy_evaluation(V, policy, gamma, theta)
        stable = policy_improvent(V, policy, gamma)
        if stable:
            break
    return V, policy


env = gym.make("Taxi-v3", render_mode="human")
env = env.unwrapped

V, policy = policy_iteration(gamma = 0.1, max_iteration = 10000, theta = 0.0001)
state = env.reset()
state = state[0]
rewards = 0
steps = 0
while True:
    env.render()
    action = np.argmax(policy[state])
    next_state, reward, done, MDP, info = env.step(action)
    rewards += reward
    steps += 1
    if done:
        break
    state = next_state
print(f'total reward: {rewards}, steps: {steps}')

total reward: 5, steps: 16


In [29]:
def value_iteration(gamma, max_iteration, theta):
    V = np.zeros(env.observation_space.n)
    for i in range(max_iteration):
        delta = 0
        for s in range(env.observation_space.n):
            q_s = np.zeros(env.action_space.n)
            for a in range(env.action_space.n):
                for prob, next_state, reward, done in env.P[s][a]:
                    if done:
                        q_s[a] += prob * reward
                    else:
                        q_s[a] += (prob * (reward + gamma * V[next_state]))
            best_value = max(q_s)
            delta = max(delta, np.abs(best_value - V[s]))
            V[s] = best_value
        if delta < theta:
            break
    policy = get_policy(V, gamma)
    return V, policy
    
def get_policy(V, gamma):
    policy = np.zeros((env.observation_space.n, env.action_space.n))
    for s in range(env.observation_space.n):
        q_s = np.zeros(env.action_space.n)
        for a in range(env.action_space.n):
            for prob, next_state, reward, done in env.P[s][a]:
                if done:
                    q_s[a] += prob * reward
                else:
                    q_s[a] += (prob * (reward + gamma * V[next_state]))
        best_action = np.argmax(q_s)
        policy[s] = np.eye(env.action_space.n)[best_action]
    return policy

import gym
import numpy as np

env = gym.make("Taxi-v3", render_mode="human")
env = env.unwrapped

V, policy = value_iteration(gamma = 1.0, max_iteration = 10000, theta = 0.0001)
state = env.reset()
state = state[0]
rewards = 0
steps = 0
while True:
    env.render()
    action = np.argmax(policy[state])
    next_state, reward, done, MDP, info = env.step(action)
    rewards += reward
    steps += 1
    if done:
        break
    state = next_state
print(f'total reward: {rewards}, steps: {steps}')

total reward: 10, steps: 11


In [26]:
#Compare
p1 = policy_iteration(gamma = 0.99, max_iteration = 10000, theta = 0.0001)[1] 
p2 = value_iteration(gamma = 1.0, max_iteration = 10000, theta = 0.0001)[1] 
print(np.all(p1 == p2)) # True

True
